# Track Pattern Recognition using Nearest Hit Approach

## Introduction

Track pattern recognition is an early step of the reconstruction of data coming from a particle detector. It recognizes tracks among the subdetectors hits. Reconstructed track parameters allow to estimate the particle deviation in a magnetic field, and thus reconstruct its charge and momentum. This information is used for the reconstruction of the decay vertex, to identify the mother particle and for further particle identification.

There is wide variety of the track pattern recognition methods. They differ in how they process the hits, what kind of tracks they are able to recognize and which requirements these tracks should satisfy. Therefore, specifics of an experiment and the detector geometry affect the tracking performance and track pattern recognition methods should be adapted to it accordingly.

In this notebook a track pattern recognition for a 2D detector with circular geometry and uniform magnetic field is considered. The detector schema with hits and tracks of an event is shown in the figure below. The challenge is to recognize tracks of an event with the highest efficiecny. It supposed that one hit can belong to only one track. 

<img src="pic/detector.png" /> <br>

## About this notebook

This notebook demonstrate how nearest hit approach can be used for track pattern recognition. The notebook describes input data, the track pattern recognition method and qualyti metrics, and shows how to use them.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas
import numpy

# Input data

In [2]:
data = pandas.read_csv('hits_1000.csv', index_col=False)
#data = data[data.event.values < 100]

data.head()

,event,particle,layer,iphi,x,y
0,0,0,5,39276,55.103343,-401.233874
1,0,6,5,22685,-381.682239,135.438799
2,0,3,3,6082,160.995866,139.460859
3,0,5,2,27787,-35.433651,-150.895515
4,0,5,1,15230,-19.627350,-82.702885


# Nearest Hit

In this method a track is grown up hit by hit. On each step the nearest hit is added to the track. The method has the following steps:

* Take a hit from the first detector layer. Add it to a track.
* Find the nearest hit from the next detector layer. Add it to the track.
* Repeat for other layers.

Please, look the method script for details.

## Data Preparation

In [3]:
X = data[[u'event', u'layer', u'iphi', u'x', u'y']].values
y = data[u'particle'].values

## Track Pattern Recognition

In [4]:
from nearest_hit import NearestHit

nh = NearestHit(min_cos_value=0.9)

nh.fit(X, y)
y_reco = nh.predict(X)

## Quality metrics

In [5]:
from metrics import RecognitionQuality

rq = RecognitionQuality(track_eff_threshold=0.8, min_hits_per_track=4)
report_event, report_tracks = rq.calculate(X, y, y_reco)

In [6]:
report_event.mean(axis=0)

Event                       499.500000
ReconstructionEfficiency      0.786045
GhostRate                     0.187597
CloneRate                     0.000000
AvgTrackEfficiency            0.847078
dtype: float64